In [2]:
import scipy as sc
import numpy as np
!pip install gpxpy
import gpxpy
import requests
import urllib
import pandas as pd
import math
!pip install geopy

In [3]:
# points must have elevation data established to call this method
from geopy import distance
def distanceBetweenPoints(p1, p2):
    flat_distance = distance.distance(p1[:2], p2[:2]).km
    # print(flat_distance)
    euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
    return euclidian_distance

In [4]:
# coordinates with known elevation 
# these are the coordinates we want to get elevations from
lat = [48.633, 48.733, 45.1947, 45.1962]
lon = [-93.9667, -94.6167, -93.3257, -93.2755]

# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': lon
})

In [10]:


# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'



def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])        

    df['elev_meters'] = elevations

elevation_function(df, 'lat', 'lon')
df.head()

,lat,lon,elev_meters
0,48.6330,-93.9667,341.14
1,48.7330,-94.6167,328.80
2,45.1947,-93.3257,262.68
3,45.1962,-93.2755,272.64


In [15]:
# list of point objects
points = df.to_records()
for p in points:
    print(p)

(0, 48.633, -93.9667, 341.14)
(1, 48.733, -94.6167, 328.8)
(2, 45.1947, -93.3257, 262.68)
(3, 45.1962, -93.2755, 272.64)


In [14]:
distances = []
p1 = points[0]
p2 = points[1]
for p in points:
    distances.append(distanceBetweenPoints(p1, p2))
    p1 = p2
    p2 = p

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed